En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

# DATA ENGINEER CHALLENGE

Para el desarrollo de este challenge estoy utilizando Python 3.12.1

## Descomprimir el archivo

In [1]:
import os
from utils import unzip_json
file_path = "datasets/tweets.json.zip"
output_dir = "datasets/"
output_file_path = "datasets/farmers-protest-tweets-2021-2-4.json"
if not os.path.exists(output_file_path):
    unzip_json(file_path,output_dir)

## Q1 memory

### Pandas
Primero se intentó agregar todo el json en un dataframe en pandas, reduciendo el uso de memoria utilizando el chunksize y solo dejando las columnas que eran necesarias. Se probaron con distintos chunksize para ver el cambio del uso de la memoria.

In [2]:
from q1_memory import q1_memory_pandas

In [3]:
result = q1_memory_pandas(output_file_path,chunksize=1000)

Filename: /home/admintdp/Documentos/personalProyects/challenge_DE/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
   109 159.4141 MiB 159.4141 MiB           1   @profile(precision=4)
   110                                         def q1_memory_pandas(file_path: str,chunksize=1000) -> List[Tuple[datetime.date, str]]:
   111 159.5391 MiB   0.1250 MiB           1       tweets = pd.read_json(file_path, lines=True,chunksize=chunksize,convert_dates=True)
   112 159.5391 MiB   0.0000 MiB           1       procced_tweets = map(process_tweets, tweets)
   113 192.0273 MiB  32.4883 MiB           1       result = reduce(add, procced_tweets)
   114 192.2773 MiB   0.2500 MiB           1       top_dates = result.groupby(['date']).sum().sort_values(ascending=False).head(10).index.values.tolist()
   115 192.2773 MiB   0.0000 MiB           1       users = []
   116 192.2773 MiB   0.0000 MiB           1       result = result.reset_index()    
   117 192.4023 MiB   0.0000 M

In [4]:
result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [4]:
result = q1_memory_pandas(output_file_path,chunksize=2000)

Filename: /home/admintdp/Documentos/personalProyects/challenge_DE/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
   109 157.2891 MiB 157.2891 MiB           1   @profile(precision=4)
   110                                         def q1_memory_pandas(file_path: str,chunksize=1000) -> List[Tuple[datetime.date, str]]:
   111 157.2891 MiB   0.0000 MiB           1       tweets = pd.read_json(file_path, lines=True,chunksize=chunksize,convert_dates=True)
   112 157.2891 MiB   0.0000 MiB           1       procced_tweets = map(process_tweets, tweets)
   113 214.9414 MiB  57.6523 MiB           1       result = reduce(add, procced_tweets)
   114 215.3164 MiB   0.3750 MiB           1       top_dates = result.groupby(['date']).sum().sort_values(ascending=False).head(10).index.values.tolist()
   115 215.3164 MiB   0.0000 MiB           1       users = []
   116 215.3164 MiB   0.0000 MiB           1       result = result.reset_index()    
   117 215.3164 MiB   0.0000 M

In [6]:
result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [3]:
result = q1_memory_pandas(output_file_path,chunksize=500)

Filename: /home/admintdp/Documentos/personalProyects/challenge_DE/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
   109 152.0859 MiB 152.0859 MiB           1   @profile(precision=4)
   110                                         def q1_memory_pandas(file_path: str,chunksize=1000) -> List[Tuple[datetime.date, str]]:
   111 152.0859 MiB   0.0000 MiB           1       tweets = pd.read_json(file_path, lines=True,chunksize=chunksize,convert_dates=True)
   112 152.0859 MiB   0.0000 MiB           1       procced_tweets = map(process_tweets, tweets)
   113 175.2500 MiB  23.1641 MiB           1       result = reduce(add, procced_tweets)
   114 175.5000 MiB   0.2500 MiB           1       top_dates = result.groupby(['date']).sum().sort_values(ascending=False).head(10).index.values.tolist()
   115 175.5000 MiB   0.0000 MiB           1       users = []
   116 175.5000 MiB   0.0000 MiB           1       result = result.reset_index()    
   117 175.5000 MiB   0.0000 M

In [4]:
result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Dataframe con tipos definidos pero con pérdida de datos no necesarios.

In [4]:
from q1_memory import  q1_memory_pandas_types
result = q1_memory_pandas_types(output_file_path,chunksize=1000)

Filename: /home/admintdp/Documentos/personalProyects/challenge_DE/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    71 153.3594 MiB 153.3594 MiB           1   @profile(precision=4)
    72                                         def q1_memory_pandas_types(file_path: str,chunksize=1000) -> List[Tuple[datetime.date, str]]:
    73 153.3594 MiB   0.0000 MiB          19       tweets = pd.read_json(file_path, lines=True,chunksize=chunksize,convert_dates=True,dtype={
    74 153.3594 MiB   0.0000 MiB           1           "url":"bool",
    75 153.3594 MiB   0.0000 MiB           1           "date":"datetime64[ns]",
    76 153.3594 MiB   0.0000 MiB           1           "content":"bool",
    77 153.3594 MiB   0.0000 MiB           1           "renderedContent": "bool",
    78 153.3594 MiB   0.0000 MiB           1           "id": "bool",
    79 153.3594 MiB   0.0000 MiB           1           "user":"object",
    80 153.3594 MiB   0.0000 MiB           1           "o

In [5]:
result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

### Polars
Una alternativa a pandas la cual utiliza código de bajo nivel para optimizar la memoria y el tiempo. Gracias a sus lazy dataframes le permite tener mejores resultados que pandas. Sin embargo al utilizar hilos paralelos puede consumir mucha más memoria.

In [2]:
from q1_memory import q1_memory_polars
result = q1_memory_polars(output_file_path)

Filename: /home/admintdp/Documentos/personalProyects/challenge_DE/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
   142    152.1 MiB    152.1 MiB           1   @profile 
   143                                         def read_lazy_json(file_path:str):
   144    154.9 MiB      2.9 MiB           1       tweets = pl.scan_ndjson(file_path,low_memory=True)
   145    156.2 MiB      0.5 MiB           2       tweets = tweets.select(
   146    155.7 MiB      0.8 MiB           1           [pl.col("date").cast(pl.Datetime).cast(pl.Date), pl.col("user").struct.field("username").alias("user")]
   147                                             )
   148    156.3 MiB      0.1 MiB           2       tweets = tweets.group_by(["date","user"]).agg(
   149    156.2 MiB      0.0 MiB           1             pl.col("date").count().alias("count"),
   150                                                 )    
   151    156.3 MiB      0.0 MiB           3       tweets = tweets.sort

In [3]:
result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 23), 'Surrypuria')]

### Linea por linea

In [2]:
from q1_memory import q1_memory_line_by_line

result = q1_memory_line_by_line(output_file_path)

Filename: /home/admintdp/Documentos/personalProyects/challenge_DE/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    23 152.2227 MiB 152.2227 MiB           1   @profile(precision=4)
    24                                         def q1_memory_line_by_line(file_path: str) -> List[Tuple[datetime.date, str]]:
    25                                             # We need to optimize the memory
    26                                             # Read line by line using json
    27 156.3477 MiB   0.0000 MiB          27       tweets_dict = defaultdict(lambda: defaultdict(int))
    28 157.0977 MiB   0.1250 MiB           2       with open(file_path, 'r', encoding='utf-8') as file:
    29                                                 # Read line by line
    30 157.0977 MiB   0.2500 MiB      117408           for line in file:
    31                                                     # Load the json
    32 157.0977 MiB   3.2500 MiB      117407               twee

In [3]:
result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [2]:
from q1_memory import q1_memory_line_by_line_alt

result = q1_memory_line_by_line_alt(output_file_path)

Filename: /home/admintdp/Documentos/personalProyects/challenge_DE/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    47 152.2734 MiB 152.2734 MiB           1   @profile(precision=4)
    48                                         def q1_memory_line_by_line_alt(file_path: str) -> List[Tuple[datetime.date, str]]:
    49                                             # We need to optimize the memory
    50                                             # Read line by line using json
    51 156.2734 MiB   0.0000 MiB          27       tweets_dict = defaultdict(lambda: defaultdict(int))
    52 156.8984 MiB   0.0000 MiB           2       with open(file_path, 'r', encoding='utf-8') as file:
    53                                                 # Read line by line
    54 156.8984 MiB   0.2500 MiB      117408           for line in file:
    55                                                     # Load the json
    56 156.8984 MiB   3.1250 MiB      117407               

### Linea por linea con streaming

In [2]:
from q1_memory import q1_memory_ijson
result = q1_memory_ijson(output_file_path)

Filename: /home/admintdp/Documentos/personalProyects/challenge_DE/src/q1_memory.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    42 152.0273 MiB 152.0273 MiB           1   @profile(precision=4)   
    43                                         def q1_memory_ijson(file_path: str) -> List[Tuple[datetime.date, str]]:
    44                                             # We need to optimize the memory
    45                                             # So we need to read the file line by line or on streaming mode
    46                                             # Read line by line using json
    47                                             
    48 156.5273 MiB   0.0000 MiB          27       tweets_dict = defaultdict(lambda: defaultdict(int))
    49 157.1523 MiB   0.0000 MiB           2       with open(file_path, 'r', encoding='utf-8') as file:
    50                                                 # Read line by line
    51 157.1523 MiB   0.0000 MiB      117408    

In [3]:
result

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]